In [11]:
import pandas as pd
import ast
from tqdm import tqdm

def truncate_and_recalc_summaries(df: pd.DataFrame, method: str, x: int = 3) -> pd.DataFrame:
    """
    Dynamically truncate each row’s Docs & Action according to one of 5 methods:

       1: L = t//2
       2: L = avg( t//2 , mid_mean )
       3: L = mid_mean = (t//2 + (t-x))//2
       4: L = avg( mid_mean , t-x )
       5: L = t - x

    Where t = len(Docs) for that row, and x=3 by default.
    Then recalculate Summaries and convert lists back to strings.
    
    :param df:    DataFrame with ['UserID','Docs','Action','Summaries'] columns.
                  'Docs'/'Action' can be lists or stringified lists.
    :param method: which of the five truncation methods to apply ('1'..'5').
    :param x:     offset, default=3.
    :return:      The truncated DataFrame with updated Summaries.
    """
    df_truncated = df.copy()

    tqdm.pandas(desc=f"Processing rows with method {method}")

    #converting 'Docs' and 'Action' from string to list
    def parse_as_list(x_):
        if isinstance(x_, str):
            return ast.literal_eval(x_)
        return x_

    df_truncated['Docs'] = df_truncated['Docs'].progress_apply(parse_as_list)
    df_truncated['Action'] = df_truncated['Action'].progress_apply(parse_as_list)

    # mid_mean = t//2 + (t - x)//2
    def compute_mid_mean(t):
        return ((t // 2) + (t - x)) // 2

    # function to calculate L for each row
    def dynamic_trunc_length(docs):
        t = len(docs)

        # Method 3's mid_mean used by 2 and 4
        mid_mean = compute_mid_mean(t)
        
        if method == '1':
            # L1 = t//2
            L = t // 2

        elif method == '2':
            # L2 = average( t//2 , mid_mean )
            #    = floor( ( (t//2) + mid_mean ) / 2 )
            L = ( (t // 2) + mid_mean ) // 2

        elif method == '3':
            # L3 = mid_mean
            L = mid_mean

        elif method == '4':
            # L4 = average( mid_mean , t-x )
            #     = floor( ( mid_mean + (t-x) ) / 2 )
            L = ( mid_mean + (t - x) ) // 2

        elif method == '5':
            # L5 = t - x
            L = t - x

        else:
            raise ValueError(f"Unknown method: {method}")

        if L < 0:
            L = 0
        if L > t:
            L = t

        return L

    # 3) doing the truncation
    def truncate_list(lst):
        L = dynamic_trunc_length(lst)
        return lst[:L]

    df_truncated['Docs'] = df_truncated['Docs'].progress_apply(truncate_list)
    df_truncated['Action'] = df_truncated['Action'].progress_apply(truncate_list)

    # 4) Recalculate Summaries
    df_truncated['Summaries'] = df_truncated['Action'].progress_apply(
        lambda actions: sum(a.lower() in ['gen_summ', 'summ_gen'] for a in actions)
    )

    # conversion of 'Docs' and 'Action' back to string
    df_truncated['Docs'] = df_truncated['Docs'].progress_apply(str)
    df_truncated['Action'] = df_truncated['Action'].progress_apply(str)

    return df_truncated
